In [1]:
import numpy as np
from scipy import spatial

In [3]:
embeddings_dict = {}
with open("embeddings/glove/glove.6B.50d.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [10]:
embeddings_dict["and"]

array([ 0.26818 ,  0.14346 , -0.27877 ,  0.016257,  0.11384 ,  0.69923 ,
       -0.51332 , -0.47368 , -0.33075 , -0.13834 ,  0.2702  ,  0.30938 ,
       -0.45012 , -0.4127  , -0.09932 ,  0.038085,  0.029749,  0.10076 ,
       -0.25058 , -0.51818 ,  0.34558 ,  0.44922 ,  0.48791 , -0.080866,
       -0.10121 , -1.3777  , -0.10866 , -0.23201 ,  0.012839, -0.46508 ,
        3.8463  ,  0.31362 ,  0.13643 , -0.52244 ,  0.3302  ,  0.33707 ,
       -0.35601 ,  0.32431 ,  0.12041 ,  0.3512  , -0.069043,  0.36885 ,
        0.25168 , -0.24517 ,  0.25381 ,  0.1367  , -0.31178 , -0.6321  ,
       -0.25028 , -0.38097 ], dtype=float32)